In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import numpy as np
from matplotlib import pyplot as plt
import arrayfire as af
af.set_backend('cpu')

from dg_maxwell import utils
from dg_maxwell import lagrange

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

In [2]:
N_LGL = 8
xi_LGL = lagrange.LGL_points(N_LGL)
L_basis_poly1d, L_basis_af = lagrange.lagrange_polynomials(xi_LGL)
L_basis_af = af.np_to_af_array(L_basis_af)

In [3]:
# Polyval 2D
polynomials = L_basis_af

x = utils.linspace(-1, 1, 10)
y = utils.linspace(-1, 1, 2)
y = af.Array([0.2])
print(y)
x, y = utils.af_meshgrid(x, y)

print(x)
print(y)

arrayfire.Array()
Type: float

[1 1 1 1]
    0.2000 

arrayfire.Array()
Type: double

[1 10 1 1]
   -1.0000    -0.7778    -0.5556    -0.3333    -0.1111     0.1111     0.3333     0.5556     0.7778     1.0000 

arrayfire.Array()
Type: float

[1 10 1 1]
    0.2000     0.2000     0.2000     0.2000     0.2000     0.2000     0.2000     0.2000     0.2000     0.2000 



In [4]:
N = int(polynomials.shape[1])
print(utils.shape(x))
print(y.shape)
x = af.reorder(x, d0 = 2, d1 = 1, d2 = 0)
y = af.reorder(y, d0 = 2, d1 = 1, d2 = 0)
x = af.tile(x, d0 = N)
y = af.tile(y, d0 = N)
print(x.shape)
print(y.shape)

power_x = af.tile(af.range(N), d0 = 1, d1 = utils.shape(x)[1], d2 = utils.shape(x)[2])
power_y = af.flip(power_x, dim = 0)

x_power = x ** power_x
y_power = y ** power_y

xy = x_power * y_power
print(af.tile(polynomials, d0 = 1, d1 = 1, d2 = utils.shape(xy)[2]).shape)
print(xy.shape)
print(af.tile(polynomials, d0 = 1, d1 = 1, d2 = utils.shape(xy)[2]).shape)
polyval = utils.matmul_3D(af.tile(polynomials, d0 = 1, d1 = 1, d2 = utils.shape(xy)[2]), xy)

[1, 10, 1, 1]
(1, 10)
(8, 10)
(8, 10)
(8, 8)
(8, 10)
(8, 8)


In [5]:
print(polyval)

arrayfire.Array()
Type: double

[8 10 1 1]
    0.0034    -0.0032    -0.0009     0.0000    -0.0000    -0.0000    -0.0000     0.0004     0.0019    -0.0023 
   -0.0099     0.0094     0.0027    -0.0000     0.0001     0.0000     0.0000    -0.0011    -0.0051     0.0062 
    0.0216    -0.0219    -0.0076    -0.0005    -0.0001    -0.0000    -0.0000     0.0018     0.0086    -0.0107 
   -0.9992    -0.1683    -0.0128     0.0000     0.0001     0.0001     0.0000    -0.0034    -0.0173     0.0227 
   -0.0227     0.0173     0.0034    -0.0000    -0.0001    -0.0001    -0.0000     0.0128     0.1683     0.9992 
    0.0107    -0.0086    -0.0018     0.0000     0.0000     0.0001     0.0005     0.0076     0.0219    -0.0216 
   -0.0062     0.0051     0.0011    -0.0000    -0.0000    -0.0001     0.0000    -0.0027    -0.0094     0.0099 
    0.0023    -0.0019    -0.0004     0.0000     0.0000     0.0000    -0.0000     0.0009     0.0032    -0.0034 


